In [1]:
import numpy as np
import pandas as pd
#col_array = ['label', 'label2' 'label_id', 'speed', 'heading_x', 'heading_y', 'heading_z', 'acceleration_x', 'acceleration_y', 'acceleration_z', 'gyroscope_x', 'gyroscope_y', 'gyroscope_z', 'magnetic_field_x', 'magnetic_field_y', 'magnetic_field_z', 'pressure_hPa', 'log_timestamp']
col_array = ['label_id', 'acceleration_x', 'acceleration_y', 'acceleration_z', 'gyroscope_x', 'gyroscope_y', 'gyroscope_z', 'magnetic_field_x', 'magnetic_field_y', 'magnetic_field_z']
path = "../posmo/experiments/data/posmo_segments/v8/" 
def read_values(csv, col_array, path="../posmo/experiments/data/posmo_segments/v8/"):
    return pd.read_csv(path + csv).loc[:, col_array] 

In [2]:
import tensorflow as tf

In [3]:
import glob
files = []
for fname in glob.glob("../posmo/experiments/data/posmo_segments/v8/*.csv"):
    files.append(fname)
for file in files:
    if file in glob.glob("../posmo/experiments/data/posmo_segments/v8/Other*"):
        files.remove(file) 

In [4]:
'''
bike1 = read_values("Bike-20180425T600 PM-aelvyr@gmail.com-8DD36F40-3778-454F-A8D8-E330C6F39830.csv", col_array)
bike2 = read_values("Bike-20180426T518 PM-aelvyr@gmail.com-33A201D1-213B-4B6B-A555-84A9CF015E87.csv", col_array)
bike3 = read_values("Bike-20180426T527 PM-aelvyr@gmail.com-8623C55E-8B27-4D36-93C5-7F91F53CFF6C.csv", col_array)
bike4 = read_values("Bike-20180427T149 PM-aelvyr@gmail.com-1222C4CE-8523-45AD-A22B-030C80EEEDDC.csv", col_array)
bike5 = read_values("Bike-20180427T1218 AM-aelvyr@gmail.com-09521095-A9D3-4662-A914-7DCBC1422947.csv", col_array)
bus1 = read_values("Bus-20180423T0929-djordje@datamap.io-F068A314-DC4D-4FB0-9D97-E79929993E41.csv", col_array)
bus2 = read_values("Bus-20180423T2150-djordje@datamap.io-DEB56810-DB1E-4E14-A144-130B52E0F405.csv", col_array)
car1 = read_values("Car-20180420T0000-djordje@datamap.io-B2876225-A37C-4A72-B045-790E5437B114.csv", col_array)
car2 = read_values("Car-20180420T0912-djordje@datamap.io-37B35625-AAF7-4217-9535-38D6BCF3FF93.csv", col_array)
car3 = read_values("Car-20180420T1328-roger@datamap.io-A3DB32B6-A6AC-4BD9-88EF-8AFFCFCCEDC1.csv", col_array)
flat1 = read_values("Other-Flat-20180424T1445-roger@datamap.io-1A6D2916-7C62-48D4-8CB9-9CBBA9FD7E20.csv", col_array)
flat2 = read_values("Other-flat-reversed-20180424T1538-roger@datamap.io-7419A222-F45B-415A-A3DF-08691E85B99A.csv", col_array)
standing = read_values("Other-InPocket-20180424T1446-roger@datamap.io-6DA03A16-DE1F-4508-BFC8-E8403F621954.csv", col_array)
scooter = read_values("Other-Scooter-20180422T2049-roger@datamap.io-0C33D54C-1B7B-4C45-92D3-2F89A7A82C16.csv", col_array)
tram1 = read_values("Tram-20180426T1825-roger@datamap.io-7C72A146-5733-446A-8418-5126D1BB74DE.csv", col_array)
tram2 = read_values("Tram-20180427T1806-roger@datamap.io-9D3D94ED-4E6B-4382-87A0-0F9A3BA4AA82.csv", col_array)
train = read_values("Train-20180420T1731-roger@datamap.io-E15B0F2F-910B-49CC-A96D-D54276492C97.csv", col_array)
walk1 = read_values("Walk-20180418T1004-roger@datamap.io-D7CA4A5B-0BDC-4E28-9BF4-3DD46B6F91AC.csv", col_array)
walk2 = read_values("Walk-20180418T1507-roger@datamap.io-F3AD9D9D-6B43-4AD1-885B-EAFF3E531A25.csv", col_array)
walk3 = read_values("Walk-20180418T1753-roger@datamap.io-AE01B243-595E-4F19-BAE3-8068038A1AE0.csv", col_array)
walk4 = read_values("Walk-20180419T0011-roger@datamap.io-50A0AB4B-4DB1-4231-8BA6-179DF3EFA381.csv", col_array)
walk5 = read_values("Walk-20180419T1025-roger@datamap.io-9F5D6118-143B-4625-8CD2-A94C5E7E62A0.csv", col_array)
'''


'\nbike1 = read_values("Bike-20180425T600 PM-aelvyr@gmail.com-8DD36F40-3778-454F-A8D8-E330C6F39830.csv", col_array)\nbike2 = read_values("Bike-20180426T518 PM-aelvyr@gmail.com-33A201D1-213B-4B6B-A555-84A9CF015E87.csv", col_array)\nbike3 = read_values("Bike-20180426T527 PM-aelvyr@gmail.com-8623C55E-8B27-4D36-93C5-7F91F53CFF6C.csv", col_array)\nbike4 = read_values("Bike-20180427T149 PM-aelvyr@gmail.com-1222C4CE-8523-45AD-A22B-030C80EEEDDC.csv", col_array)\nbike5 = read_values("Bike-20180427T1218 AM-aelvyr@gmail.com-09521095-A9D3-4662-A914-7DCBC1422947.csv", col_array)\nbus1 = read_values("Bus-20180423T0929-djordje@datamap.io-F068A314-DC4D-4FB0-9D97-E79929993E41.csv", col_array)\nbus2 = read_values("Bus-20180423T2150-djordje@datamap.io-DEB56810-DB1E-4E14-A144-130B52E0F405.csv", col_array)\ncar1 = read_values("Car-20180420T0000-djordje@datamap.io-B2876225-A37C-4A72-B045-790E5437B114.csv", col_array)\ncar2 = read_values("Car-20180420T0912-djordje@datamap.io-37B35625-AAF7-4217-9535-38D6BCF3F

In [5]:
label_dict = {'Bike': 0, 'Bus': 1, 'Car': 2, 'Tram': 3, 'Train': 4, 'Walk': 5}

In [4]:
def feature_extraction(df, f_train, l_train, f_test, l_test, seq_len=250):
    df_np = df.as_matrix()
    df_np = df_np[:(df_np.shape[0] - df_np.shape[0] % seq_len), :]
    df_np = df_np.reshape(int(df_np.shape[0]/seq_len), seq_len, df_np.shape[1])
    for i in range(df_np.shape[0]):
        label = df_np[i, 0, 0]
        feature = np.expand_dims(df_np[i, :, 1:10], axis=0)
        if i % 10 == 0:
            l_test = np.append(l_test, label)
            f_test = np.concatenate([f_test, feature], axis=0)
        else:
            l_train = np.append(l_train, label)
            f_train = np.concatenate([f_train, feature], axis=0)
    #feature = df_np[:, :, 1:18]
    #len = feature.shape[0]
    #n = int((len - len % 10) / 10)
    #feature_test = df_np[:n, :, 1:18]
    #feature_train = df_np[n:, :, 1:18]
    #f_test = np.concatenate([f_test, feature_test], axis=0)
    #f_train = np.concatenate([f_train, feature_train], axis=0)
    return f_train, l_train, f_test, l_test

In [5]:
def return_shapes():
    return f_train.shape, l_train.shape, f_test.shape, l_test.shape

In [6]:
f_train = np.empty([1, 250, 9])
l_train = np.array([])
f_test = np.empty([1, 250, 9])
l_test = np.array([])

In [8]:
for file in files:
    df = read_values(file, col_array, path="")
    f_train, l_train, f_test, l_test = feature_extraction(df, f_train, l_train, f_test, l_test)

In [ ]:
"""
f_train, l_train, f_test, l_test = feature_extraction(bike1, f_train, l_train, f_test, l_test)
f_train, l_train, f_test, l_test = feature_extraction(bike2, f_train, l_train, f_test, l_test)
f_train, l_train, f_test, l_test = feature_extraction(bike3, f_train, l_train, f_test, l_test)
f_train, l_train, f_test, l_test = feature_extraction(bike4, f_train, l_train, f_test, l_test)
f_train, l_train, f_test, l_test = feature_extraction(bike5, f_train, l_train, f_test, l_test)
"""

In [9]:
f_train = f_train[1:, :, :]
f_test = f_test[1:, :, :]

In [10]:
return_shapes()

((20537, 250, 9), (20537,), (2358, 250, 9), (2358,))

In [9]:
#f_train, l_train, f_test, l_test = feature_extraction(bus1, f_train, l_train, f_test, l_test)
#f_train, l_train, f_test, l_test = feature_extraction(bus2, f_train, l_train, f_test, l_test)

In [10]:
#return_shapes()

((573, 250, 9), (573,), (68, 250, 9), (68,))

In [11]:
#f_train, l_train, f_test, l_test = feature_extraction(car1, f_train, l_train, f_test, l_test)
#f_train, l_train, f_test, l_test = feature_extraction(car2, f_train, l_train, f_test, l_test)
#f_train, l_train, f_test, l_test = feature_extraction(car3, f_train, l_train, f_test, l_test)

In [12]:
#return_shapes()

((1070, 250, 9), (1070,), (124, 250, 9), (124,))

In [13]:
#f_train, l_train, f_test, l_test = feature_extraction(tram1, f_train, l_train, f_test, l_test)
#f_train, l_train, f_test, l_test = feature_extraction(tram2, f_train, l_train, f_test, l_test)

In [14]:
#return_shapes()

((1165, 250, 9), (1165,), (136, 250, 9), (136,))

In [15]:
#f_train, l_train, f_test, l_test = feature_extraction(train, f_train, l_train, f_test, l_test)

In [16]:
#return_shapes()

((1260, 250, 9), (1260,), (147, 250, 9), (147,))

In [17]:
#f_train, l_train, f_test, l_test = feature_extraction(walk1, f_train, l_train, f_test, l_test)
#f_train, l_train, f_test, l_test = feature_extraction(walk2, f_train, l_train, f_test, l_test)
#f_train, l_train, f_test, l_test = feature_extraction(walk3, f_train, l_train, f_test, l_test)
#f_train, l_train, f_test, l_test = feature_extraction(walk4, f_train, l_train, f_test, l_test)
#f_train, l_train, f_test, l_test = feature_extraction(walk5, f_train, l_train, f_test, l_test)

In [18]:
#return_shapes()

((2202, 250, 9), (2202,), (254, 250, 9), (254,))

In [11]:
l_train = tf.one_hot(l_train, 14)

In [12]:
l_test = tf.one_hot(l_test, 14)

In [19]:
# parameters
# changed lr: 0.1, 0.01, 0.001, 0.0001\
# changed epochs: 100, 1000, 10000, 100000
# best result from lr: 0.01, ep: 1000
learning_rate = 0.0005
epochs = 2000

# network Parameters
n_classes = 14
dropout = 0.8

seed = 123

tf.set_random_seed(seed)

weights = {
    'wc1': tf.Variable(tf.random_normal([5, 9, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 32, 64])),
    'wc3': tf.Variable(tf.random_normal([5, 64, 128])),
    'wd1': tf.Variable(tf.random_normal([10*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bc3': tf.Variable(tf.random_normal([128])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

# changed relu to sigmoid
# (best) result from reLu 
def conv1d(x, W, b, s):
    x = tf.nn.conv1d(x, W, s, padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def conv_net(x, weights, biases, dropout):
    # Conv Layer 1
    conv1 = conv1d(x, weights['wc1'], biases['bc1'], 5)

    # Conv Layer 2
    conv2 = conv1d(conv1, weights['wc2'], biases['bc2'], 5)
    
    #Conv Layer 3
    # conv3 = conv1d(conv2, weights['wc3'], biases['bc3'], 1)
    

    # Fully connected layer 
    # Changed relu to sigmoid
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.sigmoid(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction 
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

x = tf.placeholder(tf.float32, [None, 250, 9])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        sess.run(optimizer, feed_dict={
            x: f_train,
            y: l_train.eval(),
            keep_prob: dropout})

            # Calculate batch loss and accuracy
        loss = sess.run(cost, feed_dict={
            x: f_train,
            y: l_train.eval(),
            keep_prob: 1.})

        print('Epoch {:>2}-'
            'Loss: {:>10.4f}'.format(
            epoch + 1,
            loss))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: f_test,
        y: l_test.eval(),
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1-Loss:    51.6942
Epoch  2-Loss:    49.0757
Epoch  3-Loss:    45.6509
Epoch  4-Loss:    43.7992
Epoch  5-Loss:    41.6886
Epoch  6-Loss:    40.1330
Epoch  7-Loss:    38.6049
Epoch  8-Loss:    36.9378
Epoch  9-Loss:    36.0681
Epoch 10-Loss:    34.8903
Epoch 11-Loss:    33.6272
Epoch 12-Loss:    32.5347
Epoch 13-Loss:    31.6643
Epoch 14-Loss:    30.8735
Epoch 15-Loss:    29.7632
Epoch 16-Loss:    28.9796
Epoch 17-Loss:    28.3103
Epoch 18-Loss:    27.6892
Epoch 19-Loss:    27.1041
Epoch 20-Loss:    26.1096
Epoch 21-Loss:    25.5710
Epoch 22-Loss:    24.9325
Epoch 23-Loss:    24.4280
Epoch 24-Loss:    24.0572
Epoch 25-Loss:    23.4647
Epoch 26-Loss:    22.8892
Epoch 27-Loss:    22.0913
Epoch 28-Loss:    21.5627
Epoch 29-Loss:    21.0387
Epoch 30-Loss:    20.4674
Epoch 31-Loss:    20.1006
Epoch 32-Loss:    19.5421
Epoch 33-Loss:    19.0405
Epoch 34-Loss:    18.6748
Epoch 35-Loss:    18.3499
Epoch 36-Loss:    18.1861
Epoch 37-Loss:    17.8631
Epoch 38-Loss:    17.3691
Epoch 39-Los

Epoch 309-Loss:     5.3945
Epoch 310-Loss:     5.3098
Epoch 311-Loss:     5.1254
Epoch 312-Loss:     5.1359
Epoch 313-Loss:     4.9295
Epoch 314-Loss:     5.0106
Epoch 315-Loss:     5.0940
Epoch 316-Loss:     5.0966
Epoch 317-Loss:     5.0661
Epoch 318-Loss:     5.0627
Epoch 319-Loss:     5.1066
Epoch 320-Loss:     5.1820
Epoch 321-Loss:     5.3262
Epoch 322-Loss:     5.1485
Epoch 323-Loss:     5.1459
Epoch 324-Loss:     5.1968
Epoch 325-Loss:     5.1740
Epoch 326-Loss:     5.2237
Epoch 327-Loss:     5.0686
Epoch 328-Loss:     5.0118
Epoch 329-Loss:     4.9419
Epoch 330-Loss:     5.0130
Epoch 331-Loss:     5.1008
Epoch 332-Loss:     5.0885
Epoch 333-Loss:     5.1480
Epoch 334-Loss:     5.2188
Epoch 335-Loss:     5.1138
Epoch 336-Loss:     5.0950
Epoch 337-Loss:     5.0395
Epoch 338-Loss:     4.9802
Epoch 339-Loss:     5.0155
Epoch 340-Loss:     5.1318
Epoch 341-Loss:     4.9975
Epoch 342-Loss:     5.0506
Epoch 343-Loss:     4.9556
Epoch 344-Loss:     4.8846
Epoch 345-Loss:     4.8794
E

Epoch 613-Loss:     3.8297
Epoch 614-Loss:     3.8308
Epoch 615-Loss:     3.8407
Epoch 616-Loss:     3.9325
Epoch 617-Loss:     3.9167
Epoch 618-Loss:     3.9317
Epoch 619-Loss:     4.0252
Epoch 620-Loss:     4.0741
Epoch 621-Loss:     3.9644
Epoch 622-Loss:     3.8352
Epoch 623-Loss:     3.8374
Epoch 624-Loss:     3.8509
Epoch 625-Loss:     3.7992
Epoch 626-Loss:     3.7982
Epoch 627-Loss:     3.8626
Epoch 628-Loss:     3.8407
Epoch 629-Loss:     3.9816
Epoch 630-Loss:     3.9566
Epoch 631-Loss:     3.7830
Epoch 632-Loss:     3.7751
Epoch 633-Loss:     3.6638
Epoch 634-Loss:     3.5456
Epoch 635-Loss:     3.7014
Epoch 636-Loss:     3.6171
Epoch 637-Loss:     3.6112
Epoch 638-Loss:     3.7063
Epoch 639-Loss:     3.8303
Epoch 640-Loss:     3.7802
Epoch 641-Loss:     3.7949
Epoch 642-Loss:     3.6569
Epoch 643-Loss:     3.7329
Epoch 644-Loss:     3.5308
Epoch 645-Loss:     3.7505
Epoch 646-Loss:     3.5085
Epoch 647-Loss:     3.4377
Epoch 648-Loss:     3.4978
Epoch 649-Loss:     3.5031
E

Epoch 917-Loss:     3.4164
Epoch 918-Loss:     3.3006
Epoch 919-Loss:     3.1339
Epoch 920-Loss:     3.1967
Epoch 921-Loss:     3.1695
Epoch 922-Loss:     3.1589
Epoch 923-Loss:     3.1823
Epoch 924-Loss:     3.2678
Epoch 925-Loss:     3.2389
Epoch 926-Loss:     3.2798
Epoch 927-Loss:     3.1488
Epoch 928-Loss:     3.0857
Epoch 929-Loss:     3.1542
Epoch 930-Loss:     3.3447
Epoch 931-Loss:     3.3497
Epoch 932-Loss:     3.3306
Epoch 933-Loss:     3.3309
Epoch 934-Loss:     3.3754
Epoch 935-Loss:     3.3648
Epoch 936-Loss:     3.5041
Epoch 937-Loss:     3.3488
Epoch 938-Loss:     3.3630
Epoch 939-Loss:     3.4105
Epoch 940-Loss:     3.4261
Epoch 941-Loss:     3.4840
Epoch 942-Loss:     3.5003
Epoch 943-Loss:     3.6021
Epoch 944-Loss:     3.3964
Epoch 945-Loss:     3.4329
Epoch 946-Loss:     3.5294
Epoch 947-Loss:     3.5388
Epoch 948-Loss:     3.5417
Epoch 949-Loss:     3.4307
Epoch 950-Loss:     3.5198
Epoch 951-Loss:     3.3841
Epoch 952-Loss:     3.4142
Epoch 953-Loss:     3.4962
E

Epoch 1213-Loss:     2.9139
Epoch 1214-Loss:     2.9480
Epoch 1215-Loss:     2.8455
Epoch 1216-Loss:     2.8371
Epoch 1217-Loss:     2.7483
Epoch 1218-Loss:     2.7974
Epoch 1219-Loss:     2.8443
Epoch 1220-Loss:     2.8157
Epoch 1221-Loss:     2.8035
Epoch 1222-Loss:     2.8255
Epoch 1223-Loss:     2.8246
Epoch 1224-Loss:     2.8933
Epoch 1225-Loss:     2.9017
Epoch 1226-Loss:     2.8124
Epoch 1227-Loss:     2.7727
Epoch 1228-Loss:     2.8944
Epoch 1229-Loss:     2.8684
Epoch 1230-Loss:     2.8631
Epoch 1231-Loss:     2.8082
Epoch 1232-Loss:     2.6811
Epoch 1233-Loss:     2.7064
Epoch 1234-Loss:     2.7181
Epoch 1235-Loss:     2.7522
Epoch 1236-Loss:     2.7591
Epoch 1237-Loss:     2.7563
Epoch 1238-Loss:     2.7366
Epoch 1239-Loss:     2.6997
Epoch 1240-Loss:     2.7089
Epoch 1241-Loss:     2.6004
Epoch 1242-Loss:     2.6377
Epoch 1243-Loss:     2.6123
Epoch 1244-Loss:     2.5513
Epoch 1245-Loss:     2.6595
Epoch 1246-Loss:     2.6407
Epoch 1247-Loss:     2.6613
Epoch 1248-Loss:    

Epoch 1506-Loss:     2.8925
Epoch 1507-Loss:     2.9467
Epoch 1508-Loss:     2.8789
Epoch 1509-Loss:     2.8282
Epoch 1510-Loss:     2.6665
Epoch 1511-Loss:     2.6169
Epoch 1512-Loss:     2.6585
Epoch 1513-Loss:     2.6842
Epoch 1514-Loss:     2.7160
Epoch 1515-Loss:     2.7571
Epoch 1516-Loss:     2.7518
Epoch 1517-Loss:     2.6719
Epoch 1518-Loss:     2.7078
Epoch 1519-Loss:     2.6769
Epoch 1520-Loss:     2.6005
Epoch 1521-Loss:     2.5966
Epoch 1522-Loss:     2.6545
Epoch 1523-Loss:     2.6315
Epoch 1524-Loss:     2.7389
Epoch 1525-Loss:     2.7292
Epoch 1526-Loss:     2.8232
Epoch 1527-Loss:     2.8326
Epoch 1528-Loss:     2.8216
Epoch 1529-Loss:     2.8684
Epoch 1530-Loss:     2.7870
Epoch 1531-Loss:     2.9363
Epoch 1532-Loss:     2.7345
Epoch 1533-Loss:     2.8083
Epoch 1534-Loss:     2.6695
Epoch 1535-Loss:     2.5352
Epoch 1536-Loss:     2.5162
Epoch 1537-Loss:     2.4768
Epoch 1538-Loss:     2.5561
Epoch 1539-Loss:     2.5723
Epoch 1540-Loss:     2.5993
Epoch 1541-Loss:    

Epoch 1799-Loss:     2.6479
Epoch 1800-Loss:     2.6632
Epoch 1801-Loss:     2.6946
Epoch 1802-Loss:     2.6910
Epoch 1803-Loss:     2.6463
Epoch 1804-Loss:     2.6962
Epoch 1805-Loss:     2.7138
Epoch 1806-Loss:     2.7142
Epoch 1807-Loss:     2.6547
Epoch 1808-Loss:     2.6605
Epoch 1809-Loss:     2.6597
Epoch 1810-Loss:     2.6766
Epoch 1811-Loss:     2.5475
Epoch 1812-Loss:     2.5865
Epoch 1813-Loss:     2.6413
Epoch 1814-Loss:     2.7126
Epoch 1815-Loss:     2.9140
Epoch 1816-Loss:     2.8818
Epoch 1817-Loss:     2.7345
Epoch 1818-Loss:     2.7706
Epoch 1819-Loss:     2.6949
Epoch 1820-Loss:     2.8317
Epoch 1821-Loss:     2.8032
Epoch 1822-Loss:     2.6657
Epoch 1823-Loss:     2.6194
Epoch 1824-Loss:     2.6853
Epoch 1825-Loss:     2.7498
Epoch 1826-Loss:     2.7055
Epoch 1827-Loss:     2.8512
Epoch 1828-Loss:     2.8973
Epoch 1829-Loss:     3.0027
Epoch 1830-Loss:     2.8561
Epoch 1831-Loss:     2.8469
Epoch 1832-Loss:     2.7810
Epoch 1833-Loss:     2.8090
Epoch 1834-Loss:    

In [ ]:
def neural_net(activation_function, conv_layers, learning_rate, epochs, dropout, n_filters, seed):
    # network Parameters
    n_classes = 14

    tf.set_random_seed(seed)

    if conv_layers == 2:
        weights = {
            'wc1': tf.Variable(tf.random_normal([n_filters, 9, 32])),
            'wc2': tf.Variable(tf.random_normal([n_filters, 32, 64])),
            'wc3': tf.Variable(tf.random_normal([n_filters, 64, 128])),
            'wd1': tf.Variable(tf.random_normal([10*64, 1024])),
            'out': tf.Variable(tf.random_normal([1024, n_classes]))}
    elif conv_layers == 3:
        weights = {
            'wc1': tf.Variable(tf.random_normal([n_filters, 9, 32])),
            'wc2': tf.Variable(tf.random_normal([n_filters, 32, 64])),
            'wc3': tf.Variable(tf.random_normal([n_filters, 64, 128])),
            'wd1': tf.Variable(tf.random_normal([10*128, 1024])),
            'out': tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {
        'bc1': tf.Variable(tf.random_normal([32])),
        'bc2': tf.Variable(tf.random_normal([64])),
        'bc3': tf.Variable(tf.random_normal([128])),
        'bd1': tf.Variable(tf.random_normal([1024])),
        'out': tf.Variable(tf.random_normal([n_classes]))}

    # changed relu to sigmoid
    # (best) result from reLu 
    def conv1d(x, W, b, s):
        x = tf.nn.conv1d(x, W, s, padding='SAME')
        x = tf.nn.bias_add(x, b)
        return tf.nn.relu(x)

    if conv_layers == 2:
        def conv_net(x, weights, biases, dropout, activation_function):
            # Conv Layer 1
            conv1 = conv1d(x, weights['wc1'], biases['bc1'], 5)

            # Conv Layer 2
            conv2 = conv1d(conv1, weights['wc2'], biases['bc2'], 5)
    
            #Conv Layer 3
            # conv3 = conv1d(conv2, weights['wc3'], biases['bc3'], 1)
    

            # Fully connected layer 
            # Changed relu to sigmoid
            fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
            fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
            fc1 = activation_function(fc1)
            fc1 = tf.nn.dropout(fc1, dropout)

            # Output Layer - class prediction 
            out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
            return out
    elif conv_layers == 3:
        def conv_net(x, weights, biases, dropout, activation_function):
            # Conv Layer 1
            conv1 = conv1d(x, weights['wc1'], biases['bc1'], 5)

            # Conv Layer 2
            conv2 = conv1d(conv1, weights['wc2'], biases['bc2'], 5)
    
            #Conv Layer 3
            conv3 = conv1d(conv2, weights['wc3'], biases['bc3'], 1)
    

            # Fully connected layer 
            # Changed relu to sigmoid
            fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
            fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
            fc1 = activation_function(fc1)
            fc1 = tf.nn.dropout(fc1, dropout)

            # Output Layer - class prediction 
            out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
            return out

    x = tf.placeholder(tf.float32, [None, 250, 9])
    y = tf.placeholder(tf.float32, [None, n_classes])
    keep_prob = tf.placeholder(tf.float32)

    # Model
    logits = conv_net(x, weights, biases, keep_prob, activation_function)

    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

    # Accuracy
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initializing the variables
    init = tf.global_variables_initializer()

    # Launch the graph
    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(epochs):
            sess.run(optimizer, feed_dict={
                x: f_train,
                y: l_train.eval(),
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: f_train,
                y: l_train.eval(),
                keep_prob: 1.})

            if epoch % 100 == 0:
                print('Epoch {:>2} - '
                    'Loss: {:>10.4f}'.format(
                    epoch + 1,
                    loss))

    # Calculate Test Accuracy
        test_acc = sess.run(accuracy, feed_dict={
            x: f_test,
            y: l_test.eval(),
            keep_prob: 1.})
        print('Testing Accuracy: {}'.format(test_acc))

In [ ]:
neural_net(tf.nn.sigmoid, 2, 0.0005, 2000, 0.8, 5, 123)

Epoch  1 - Loss:    45.6232
Epoch 101 - Loss:     8.5770
Epoch 201 - Loss:     6.2645
Epoch 301 - Loss:     4.8543
Epoch 401 - Loss:     4.1534
Epoch 501 - Loss:     3.8986
Epoch 601 - Loss:     3.4219
Epoch 701 - Loss:     3.5755
Epoch 801 - Loss:     3.4659
Epoch 901 - Loss:     3.6292
Epoch 1001 - Loss:     3.4280
Epoch 1101 - Loss:     3.3295
Epoch 1201 - Loss:     3.2437
Epoch 1301 - Loss:     2.9707
Epoch 1401 - Loss:     2.9898
Epoch 1501 - Loss:     3.0429
Epoch 1601 - Loss:     2.8173
Epoch 1701 - Loss:     2.8700
Epoch 1801 - Loss:     2.8705
Epoch 1901 - Loss:     2.8649


In [ ]:
neural_net(tf.nn.sigmoid, 2, 0.0005, 2000, 0.8, 5, 123)